In [1]:
def smp_probe(filename: str) -> bool:
    with open(filename, "rb") as f:
        return f.read(9) == b"file=samp"


def smp_headers(filename: str):
    with open(filename, "rb") as f:
        f.seek(0)
        raw_headers = f.read(1024)
        raw_headers = raw_headers.rstrip(b'\x00')
        asc_headers = raw_headers.decode("ascii")
        asc_headers.rstrip('\x00')
        tmp = [a for a in asc_headers.split("\r\n")]
        back = -1
        while abs(back) > len(tmp) + 1:
            if tmp[back] == '=':
                break
            back -= 1
        tmp = tmp[0:back-1]
        return dict(a.split("=") for a in tmp)

In [2]:
example = "/Users/joregan/Playing/waxholm/scenes_formatted/fp2060/fp2060.11.04.smp"

In [5]:
with open(example, "rb") as egf:
    egf.seek(0)
    blah = egf.read(1024)
    a = egf.tell()
a

1024

In [ ]:
def input_args(
        save_dir,
        tensorboard_logdir,
        data,
        w2v_path,
        empty_cache_freq=0,
        best_checkpoint_metric="uer",  # UER is a character error rate, oddly named
        n_gpu=None,
        lr=5e-05,
        max_update=12000,
        warmup_updates=4000,
        hold_updates=4000,
        decay_updates=4000,
        freeze_updates=2000,  # Freeze all but the output layer
        validate_after_updates=1000,
        validate_interval=1,
        valid_subset="valid",
        max_tokens=MAX_TOKENS,   # per device. 1250000 fills up the rtx2080 11GB
        max_tokens_per_minibatch=6400000,
        mask_prob=0.65,
        mask_channel_length=64,
        mask_channel_prob=0.25,
        update_freq=None,
        layerdrop=0.1,  # Unconfirmed but I think it's layerdrop/hidden_dropout for fairseq -> huggingface
        final_lr_scale=0.05,
        final_dropout=0.1,
        dropout=0.1,
        activation_dropout=0.1,
        attention_dropout=0.1,
        random_seed=5728395,
        adam_epsilon=1e-08,
        adam_betas=(0.9, 0.98),
        weight_decay=0.0
):
    # This is pretty clunky, but still less clunky than bash imo
    n_gpu = n_gpu or torch.cuda.device_count() or 1
    update_freq = update_freq or math.ceil(max_tokens_per_minibatch / max_tokens / n_gpu)
    args_string = f"""
      {data}
      --fp16
      --save-dir {save_dir}
      --tensorboard-logdir {tensorboard_logdir}
      --distributed-world-size {n_gpu}
      --empty-cache-freq {empty_cache_freq}
      --max-sample-size {max_tokens}
      --max-tokens {max_tokens}
      --max-tokens-valid {max_tokens}
      --update-freq [{update_freq}]
      --post-process letter
      --valid-subset {valid_subset}
      --no-epoch-checkpoints
      --best-checkpoint-metric {best_checkpoint_metric}
      --num-workers 1
      --max-update {max_update}
      --sentence-avg
      --task audio_pretraining
      --arch wav2vec_ctc
      --w2v-path {w2v_path}
      --labels ltr
      --weight-decay {weight_decay}
      --apply-mask
      --mask-selection static
      --mask-other 0
      --mask-length 10
      --mask-prob {mask_prob}
      --mask-channel-selection static
      --mask-channel-other 0
      --mask-channel-length {mask_channel_length}
      --mask-channel-prob {mask_channel_prob}
      --zero-infinity
      --feature-grad-mult 0.0
      --freeze-finetune-updates {freeze_updates}
      --validate-after-updates {validate_after_updates}
      --validate-interval {validate_interval}
      --optimizer adam
      --adam-betas {adam_betas}
      --adam-eps {adam_epsilon}
      --lr {lr}
      --lr-scheduler tri_stage
      --warmup-steps {warmup_updates}
      --hold-steps {hold_updates}
      --decay-steps {decay_updates}
      --final-lr-scale {final_lr_scale}
      --final-dropout {final_dropout}
      --dropout {dropout}
      --layerdrop {layerdrop}
      --activation-dropout {activation_dropout}
      --criterion ctc
      --attention-dropout {attention_dropout}
      --seed {random_seed}
      --log-format json
      --log-interval 50
      --ddp-backend no_c10d
    """